In [ ]:
# settings for display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# turn off matplotlib figure axes, else we need to call axes('off') everytime
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.set_cmap('gray') 
mpl.rc('axes.spines',top=False,bottom=False,left=False,right=False)
mpl.rc(('xtick','ytick'),color=(1,1,1,0))
mpl.rc('font', size=15)

import warnings
warnings.filterwarnings("ignore")

# import necessarry libs
from imageio import imread
import numpy as np
from scipy.ndimage.filters import correlate, convolve
from scipy.ndimage import gaussian_filter
from numpy.linalg import eig
from numpy import diag
import time

from skimage.color import rgb2gray
from skimage.transform import resize, rotate

# define gaussian filter
# def f_gauss(size, sigma):
#     # gaussian filter
#     x, y = np.mgrid[-size[0]//2 + 1:size[0]//2 + 1, -size[1]//2 + 1:size[1]//2 + 1]
#     g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
#     return g/g.sum()

In [ ]:
%matplotlib notebook
fig,ax = plt.subplots(1,1)

for r in np.linspace(0, 360, 50):
    '''Générons une image synthétique'''
    # Image synthétique avec une arête
    img = np.zeros([512, 512])
    img[0:int(img.shape[0]/2),:] = 1
    # img = np.random.rand(512, 512)
    
    # Image synthétique avec un coin
    # img[:,0:256] = 0;
    # img = img+img.T

    # Appliquons une rotation à l'arête (pour voir l'impact sur les valeurs et
    # vecteurs propres)
    img = rotate(img, r)
    
    # garder la partie centrale seulement
    img = img[127:384, 127:384]
    
    # filtrer
    # h = f_gauss([20, 20], 3);
    # img = correlate(img, h, mode='reflect')
    img = gaussian_filter(img, 3, mode='reflect')
    
    '''
    Calculons les valeurs et vecteurs propres de la matrice de covariance des gradients de l'image
    '''
    # gradients
    [gy,gx] = np.gradient(img)
    
    gxx = np.matmul(gx.flatten().T, gx.flatten())
    gxy = np.matmul(gx.flatten().T, gy.flatten())
    gyy = np.matmul(gy.flatten().T, gy.flatten())
    
    # matrice de covariance
    M = np.asarray([[gxx, gxy], [gxy, gyy]])
    
    # valeurs et vecteurs propres
    [the_lambda, V] = eig(M)
    
    # affichons le vecteur propre correspondant à la valeur propre maximale
    mind = np.argmax(the_lambda)

    # plot
    ax.clear()
    img_ax = ax.imshow(img)
    ax.plot([128, 128+V[0,mind]*the_lambda[mind]], [128, 128+V[1,mind]*the_lambda[mind]], 'r-', linewidth=2, markersize=12);
    plt.title('Valeurs propres: [%.2f, %.2f]' % (the_lambda[0], the_lambda[1]))
    fig.canvas.draw()    
    time.sleep(0.1)